In [2]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing, tree
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, roc_curve, auc
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#from preprocessing import preprocess
plt.rcParams["figure.dpi"] = 150

In [3]:
# from collections import Counter
# import requests

# with requests.get(
#     "https://docs.google.com/spreadsheets/d/1wduqo5WyYmCpaGnE81sLNGU0VSodIekMfpmEwU0fGqs/export?format=csv") as r, open("features.csv", "wb") as f:
#     for chunk in r.iter_content():
#         f.write(chunk)

# with requests.get(
#     "https://docs.google.com/spreadsheets/d/1gvZ03uAL6THwd04Y98GtIj6SeAHiKyQY5UisuuyFSUs/export?format=csv") as r, open("target.csv", "wb") as f:
#     for chunk in r.iter_content():
#         f.write(chunk)

In [4]:
df_features = pd.read_csv("features.csv")
df_target = pd.read_csv("target.csv")
df = df_features.merge(df_target, left_on='id', right_on='id')
df['direccion_viento_temprano'].value_counts()

/home/facundo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Nornoreste       12784
Norte             9453
Sureste           7437
Este              7391
Sursureste        7244
Noroeste          7003
Sur               6948
suroeste          6785
Oeste             6742
Estenoreste       6244
Estesureste       6152
Noreste           6136
Sursuroeste       6073
Oestenoroeste     5964
Oestesuroeste     5609
Name: direccion_viento_temprano, dtype: int64

In [5]:
def basic_preprocessing(df: pd.DataFrame):
    df['presion_atmosferica_tarde'] = pd.to_numeric(df['presion_atmosferica_tarde'],errors='coerce')    
    df['dia'] = pd.to_datetime(df['dia'])
    df = df.dropna(subset=['llovieron_hamburguesas_al_dia_siguiente']) 
    df = df.dropna(subset=['llovieron_hamburguesas_hoy'])
    df = df.drop(columns = ['id', 'barrio', 'mm_evaporados_agua'])
    df['mes'] = df['dia'].dt.month
    df = df.drop(columns = ['dia']) 
    df = df[df.isnull().mean(1) < 0.4]
    
    label_encoder = preprocessing.LabelEncoder()
    
    label_encoder.fit(df['llovieron_hamburguesas_hoy'])
    df['llovieron_hamburguesas_hoy'] = label_encoder.transform(df['llovieron_hamburguesas_hoy'])

    label_encoder.fit(df['llovieron_hamburguesas_al_dia_siguiente'])
    df['llovieron_hamburguesas_al_dia_siguiente'] = label_encoder.transform(df['llovieron_hamburguesas_al_dia_siguiente'])
    
    return df

In [6]:
def preprocessing1(df: pd.DataFrame):
    
    df = basic_preprocessing(df)

    df = pd.get_dummies(df, drop_first=True, dummy_na=True, columns=['direccion_viento_temprano', 'rafaga_viento_max_direccion', 'direccion_viento_tarde'])

    df['horas_de_sol'] = df['horas_de_sol'].fillna(df['horas_de_sol'].mean())
    df['nubosidad_tarde'] = df['nubosidad_tarde'].fillna(df['nubosidad_tarde'].mean()).astype('int')
    df['nubosidad_temprano'] = df['nubosidad_temprano'].fillna(df['nubosidad_temprano'].mean())
    df['presion_atmosferica_temprano'] = df['presion_atmosferica_temprano'].fillna(df['presion_atmosferica_temprano'].mean())
    df['presion_atmosferica_tarde'] = df['presion_atmosferica_tarde'].fillna(df['presion_atmosferica_tarde'].mean())
    df['rafaga_viento_max_velocidad'] = df['rafaga_viento_max_velocidad'].fillna(df['rafaga_viento_max_velocidad'].mean())
    df['humedad_tarde'] = df['humedad_tarde'].fillna(df['humedad_tarde'].mean())
    df['temperatura_tarde'] = df['temperatura_tarde'].fillna(df['temperatura_tarde'].mean())
    df['mm_lluvia_dia'] = df['mm_lluvia_dia'].fillna(df['mm_lluvia_dia'].median())
    df['velocidad_viendo_tarde'] = df['velocidad_viendo_tarde'].fillna(df['velocidad_viendo_tarde'].mean())
    df['humedad_temprano'] = df['humedad_temprano'].fillna(df['humedad_temprano'].mean())
    df['velocidad_viendo_temprano'] = df['velocidad_viendo_temprano'].fillna(df['velocidad_viendo_temprano'].mean())
    df['temperatura_temprano'] = df['temperatura_temprano'].fillna(df['temperatura_temprano'].mean())
    df['temp_min'] = df['temp_min'].fillna(df['temp_min'].mean())
    df['temp_max'] = df['temp_max'].fillna(df['temp_max'].mean())
    
    return df


In [7]:
def preprocessing2(df: pd.DataFrame):
    df = basic_preprocessing(df)


    df = df.drop(columns = ['direccion_viento_temprano', 'rafaga_viento_max_direccion', 'direccion_viento_tarde', 'mes', 'llovieron_hamburguesas_hoy']) 

    df['horas_de_sol'] = df['horas_de_sol'].fillna(df['horas_de_sol'].mean())
    df['nubosidad_tarde'] = df['nubosidad_tarde'].fillna(df['nubosidad_tarde'].mean())
    df['nubosidad_temprano'] = df['nubosidad_temprano'].fillna(df['nubosidad_temprano'].mean())
    df['presion_atmosferica_temprano'] = df['presion_atmosferica_temprano'].fillna(df['presion_atmosferica_temprano'].mean())
    df['presion_atmosferica_tarde'] = df['presion_atmosferica_tarde'].fillna(df['presion_atmosferica_tarde'].mean())
    df['rafaga_viento_max_velocidad'] = df['rafaga_viento_max_velocidad'].fillna(df['rafaga_viento_max_velocidad'].mean())
    df['humedad_tarde'] = df['humedad_tarde'].fillna(df['humedad_tarde'].mean())
    df['temperatura_tarde'] = df['temperatura_tarde'].fillna(df['temperatura_tarde'].mean())
    df['mm_lluvia_dia'] = df['mm_lluvia_dia'].fillna(df['mm_lluvia_dia'].median())
    df['velocidad_viendo_tarde'] = df['velocidad_viendo_tarde'].fillna(df['velocidad_viendo_tarde'].mean())
    df['humedad_temprano'] = df['humedad_temprano'].fillna(df['humedad_temprano'].mean())
    df['velocidad_viendo_temprano'] = df['velocidad_viendo_temprano'].fillna(df['velocidad_viendo_temprano'].mean())
    df['temperatura_temprano'] = df['temperatura_temprano'].fillna(df['temperatura_temprano'].mean())
    df['temp_min'] = df['temp_min'].fillna(df['temp_min'].mean())
    df['temp_max'] = df['temp_max'].fillna(df['temp_max'].mean())

    return df

In [8]:
def preprocessing3(df: pd.DataFrame):
    df = basic_preprocessing(df)

    df = pd.get_dummies(df, drop_first=True, dummy_na=True, columns=['direccion_viento_temprano', 'rafaga_viento_max_direccion', 'direccion_viento_tarde'])
    df = df.drop(columns = ['horas_de_sol', 'nubosidad_tarde', 'nubosidad_temprano']) 

    df['presion_atmosferica_temprano'] = df['presion_atmosferica_temprano'].fillna(df['presion_atmosferica_temprano'].mean())
    df['presion_atmosferica_tarde'] = df['presion_atmosferica_tarde'].fillna(df['presion_atmosferica_tarde'].mean())
    df['rafaga_viento_max_velocidad'] = df['rafaga_viento_max_velocidad'].fillna(df['rafaga_viento_max_velocidad'].mean())
    df['humedad_tarde'] = df['humedad_tarde'].fillna(df['humedad_tarde'].mean())
    df['temperatura_tarde'] = df['temperatura_tarde'].fillna(df['temperatura_tarde'].mean())
    df['mm_lluvia_dia'] = df['mm_lluvia_dia'].fillna(df['mm_lluvia_dia'].median())
    df['velocidad_viendo_tarde'] = df['velocidad_viendo_tarde'].fillna(df['velocidad_viendo_tarde'].mean())
    df['humedad_temprano'] = df['humedad_temprano'].fillna(df['humedad_temprano'].mean())
    df['velocidad_viendo_temprano'] = df['velocidad_viendo_temprano'].fillna(df['velocidad_viendo_temprano'].mean())
    df['temperatura_temprano'] = df['temperatura_temprano'].fillna(df['temperatura_temprano'].mean())
    df['temp_min'] = df['temp_min'].fillna(df['temp_min'].mean())
    df['temp_max'] = df['temp_max'].fillna(df['temp_max'].mean())
    
    return df

In [9]:
def preprocessing4(df: pd.DataFrame):
    df = basic_preprocessing(df)


    df = df.drop(columns = ['direccion_viento_temprano', 'rafaga_viento_max_direccion', 'direccion_viento_tarde', 'mes',
                           'horas_de_sol', 'nubosidad_tarde', 'nubosidad_temprano', 'temperatura_temprano', 'presion_atmosferica_temprano',
                           'velocidad_viendo_temprano', 'temperatura_temprano', 'temp_min', 'temp_max']) 

    df['presion_atmosferica_tarde'] = df['presion_atmosferica_tarde'].fillna(df['presion_atmosferica_tarde'].mean())
    df['rafaga_viento_max_velocidad'] = df['rafaga_viento_max_velocidad'].fillna(df['rafaga_viento_max_velocidad'].mean())
    df['humedad_tarde'] = df['humedad_tarde'].fillna(df['humedad_tarde'].mean())
    df['temperatura_tarde'] = df['temperatura_tarde'].fillna(df['temperatura_tarde'].mean())
    df['mm_lluvia_dia'] = df['mm_lluvia_dia'].fillna(df['mm_lluvia_dia'].median())
    df['velocidad_viendo_tarde'] = df['velocidad_viendo_tarde'].fillna(df['velocidad_viendo_tarde'].mean())
    df['humedad_temprano'] = df['humedad_temprano'].fillna(df['humedad_temprano'].mean())

    
    return df

In [10]:
def split(df):
    
    X = df.drop(columns=['llovieron_hamburguesas_al_dia_siguiente'])
    y = df['llovieron_hamburguesas_al_dia_siguiente']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y)

    return X_train, X_test, y_train, y_test




In [11]:
def plot_roc(_fpr, _tpr, x):

    roc_auc = auc(_fpr, _tpr)

    plt.figure(figsize=(15, 10))
    plt.plot(
        _fpr, _tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})'
    )
    plt.scatter(_fpr, x)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()



In [16]:
df1 = preprocessing1(df)
X_train, X_test, y_train, y_test = split(df1)
params = {
    'min_samples_split': np.arange(2, 10),
    'n_estimators': np.arange(5, 20),
    #'learning_rate': np.arange(0.1, 0.5),
   
    
}

clf = RandomForestClassifier(random_state=0)

rgscv = RandomizedSearchCV(
    clf, params, n_iter = 60, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8706845566305885
Best params {'n_estimators': 19, 'min_samples_split': 8}


/home/facundo/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/facundo/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/facundo/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/facundo/.local/lib/python3.8/site-packages/joblib/parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/facundo/.local/lib/python3.8/site-packages/joblib/parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/facundo/.local/lib/python3.8/site-packages/joblib/parallel.py", line 777, in _dispatch
    job = self._bac

In [18]:
df2 = preprocessing2(df)
X_train, X_test, y_train, y_test = split(df2)
params = {
    'min_samples_split': np.arange(2, 10),
    'n_estimators': np.arange(5, 20),
    #'learning_rate': np.arange(0.1, 0.5),
       
}

clf = RandomForestClassifier(random_state=0)


rgscv = RandomizedSearchCV(
    clf, params,  n_iter = 60, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8769171647933197
Best params {'n_estimators': 121, 'max_depth': 4}


In [ ]:
df3 = preprocessing3(df)
X_train, X_test, y_train, y_test = split(df3)
params = {
    'min_samples_split': np.arange(2, 10),
    'n_estimators': np.arange(5, 20),
    #'learning_rate': np.arange(0.1, 0.5),
   
    
}

clf = RandomForestClassifier(random_state=0)


rgscv = RandomizedSearchCV(
    clf, params, n_iter = 1, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

In [ ]:
df4 = preprocessing4(df)
X_train, X_test, y_train, y_test = split(df4)
params = {
    'min_samples_split': np.arange(2, 10),
    'n_estimators': np.arange(5, 20),
    #'learning_rate': np.arange(0.1, 0.5),
   
    
}

clf = RandomForestClassifier(random_state=0)

rgscv = RandomizedSearchCV(
    clf, params, n_iter = 1, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

In [ ]:
#df1 = preprocessing1(df)
X_train, X_test, y_train, y_test = split(df1)


#clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"Auc Roc: {roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])}")


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
plot_roc(fpr, tpr, thresholds)